# Working on Amazon Employee Reviews dataset

The task is to develop a system that can identify both negative and positive topics in the provided dataset (“Amazon Jan 2023.csv”). In simpler terms, your task involves the following:
1. **Sentiment Analysis:** Divide the dataset into positive and negative data. Please consider that each comment can be classified only as positive or negative.
2. **Topic Modeling:** Detect the subjects being discussed in both the positive and negative comments. Please keep in mind that each comment might cover multiple topics.
3. **Displaying the Results:** Upon executing your code, we expect to observe several additional columns integrated into the current dataset.
    * A new column labeled `Sentiment` featuring either _"Positive"_ or _"Negative"_ labels corresponding to the sentiment of each comment.
    * Separate columns for each identified topic, featuring either _"Yes"_ or _"No"_ labels to indicate whether a comment covers that specific topic or not.

In [4]:
# # Install bertopic
# !pip install bertopic

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import nltk
import gensim
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.utils import simple_preprocess

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

# Topic model
from bertopic import BERTopic

# Dimension reduction
from umap import UMAP

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# df = pd.read_csv("Data/Amazon Jan 2023.csv") # On system
df = pd.read_csv("/content/drive/MyDrive/Data_Science/Coding_Challenge/Amazon Jan 2023.csv") # Google colab
df.head()

,Comment
0,Fun and flexible work environment and an oppor...
1,"be prepared for constant changes, rules, error..."
2,Award-winning training. All the support you ne...
3,very long hours
4,Inclusive environment. Good salary and benefit...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  99 non-null     object
dtypes: object(1)
memory usage: 920.0+ bytes


In [8]:
# Check for null values
df.isnull().sum()

Comment    0
dtype: int64

In [9]:
df.loc[90, "Comment"]

'The opportunities to raise through the ladder are low and are extremely tough with very high competition.'

In [10]:
df.loc[9, "Comment"]

'Prepare to drive lots of miles for work. Drive tons of miles and far distances from pickup points at times.'

In [11]:
df.loc[50, "Comment"]

'However, I do have a really good manager which helps the situation a lot.'

In [12]:
data = df.copy()

## Sentiment Analysis Scores using Roberta

* Labels: 0 -> Negative; 1 -> Neutral; 2 -> Positive

In [13]:
# Load model and tokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [14]:
# Define a function that will generate sentiment scores
def polarity_scores_roberta(comment):
    inputs = tokenizer.encode(comment, return_tensors='pt')
    outputs = model(inputs)[0]
    scores = softmax(outputs.detach().numpy(), axis=1)[0]
    return scores

# Generate sentiment scores for each comment
data["Roberta_Sentiment_Scores"] = data["Comment"].apply(polarity_scores_roberta)
data.head()


,Comment,Roberta_Sentiment_Scores
0,Fun and flexible work environment and an oppor...,"[0.0017704904, 0.049383562, 0.948846]"
1,"be prepared for constant changes, rules, error...","[0.66478276, 0.3168306, 0.01838666]"
2,Award-winning training. All the support you ne...,"[0.010903009, 0.2204135, 0.76868343]"
3,very long hours,"[0.3425702, 0.5902841, 0.067145616]"
4,Inclusive environment. Good salary and benefit...,"[0.001945477, 0.032334775, 0.96571976]"


The code defines a function called `polarity_scores_roberta` that takes a comment as input.

Within the function, the `tokenizer` is used to encode the `comment` and return the encoded tensor.

The `model` is then used to generate sentiment scores based on the encoded tensor. The function uses the model to get outputs, applies `softmax` function to the outputs along the specified axis, and then detaches the outputs as numpy array.

In [15]:
# Define a function that will generate sentiment
def analyze_sentiment(comment):
    inputs = tokenizer(comment, return_tensors='pt', truncation=True, padding=True, max_length=512)
    logits = model(**inputs).logits
    scores = "Positive" if logits.argmax() == 2 else "Negative"
    return scores

# Generate sentiment scores for each comment
data["Sentiment"] = data["Comment"].apply(analyze_sentiment)
data.head()


,Comment,Roberta_Sentiment_Scores,Sentiment
0,Fun and flexible work environment and an oppor...,"[0.0017704904, 0.049383562, 0.948846]",Positive
1,"be prepared for constant changes, rules, error...","[0.66478276, 0.3168306, 0.01838666]",Negative
2,Award-winning training. All the support you ne...,"[0.010903009, 0.2204135, 0.76868343]",Positive
3,very long hours,"[0.3425702, 0.5902841, 0.067145616]",Negative
4,Inclusive environment. Good salary and benefit...,"[0.001945477, 0.032334775, 0.96571976]",Positive


The code first defines a function called `analyze_sentiment` that takes in a single comment as input. Inside the function, the comment is passed to a tokenizer along with some additional parameters. The tokenizer returns tokenized inputs that are suitable for input to the model.

Next, the model is called with the tokenized inputs and the `.logits` attribute is used to obtain the raw output scores. These scores are then used to determine the sentiment by comparing the index of the highest score with a condition. If the index is 2, the sentiment is considered `Positive`. Otherwise, it is considered `Negative`.

The `analyze_sentiment` function returns the sentiment as the variable scores.

In [16]:
data.loc[20, "Comment"]

"Typical Day* Challenging, but always rewarding. What I've Learned? * Overcoming the challenges and trying New things. Management* Caring, Very Helpful and wants you to succeed in every area possible. Workplace Culture* It's culture moves around Fast Pace and Productivity and Innovation. The Most Enjoyable Part of The Job* Knowing being Rewarded in your good and leveling up to become Great in your Greatness. Benefits, Dental, Medical, Visual Insurance."

In [17]:
data.loc[20, "Roberta_Sentiment_Scores"]

array([0.00258254, 0.04589062, 0.9515268 ], dtype=float32)

In [18]:
data.loc[20, "Sentiment"]

'Positive'

In [19]:
data.loc[35, "Comment"]

'Union busting is disgusting! Even though we might not work at the air hub, working people must fight back against Amazon’s bullying tactics of firing workers who dare to fight for a fair wage and safe conditions!'

In [20]:
data.loc[35, "Roberta_Sentiment_Scores"]

array([0.8766511 , 0.10989761, 0.01345122], dtype=float32)

In [21]:
data.loc[35, "Sentiment"]

'Negative'

In [22]:
data.loc[45, "Comment"]

'Competitive salaries.'

In [23]:
data.loc[45, "Roberta_Sentiment_Scores"]

array([0.2296703 , 0.6869444 , 0.08338527], dtype=float32)

In [24]:
data.loc[45, "Sentiment"]

'Negative'

In [25]:
data.loc[1, "Comment"]

'be prepared for constant changes, rules, errors, lack of support'

In [26]:
data.loc[1, "Roberta_Sentiment_Scores"]

array([0.66478276, 0.3168306 , 0.01838666], dtype=float32)

In [27]:
data.loc[1, "Sentiment"]

'Negative'

## Topic Modeling

In [28]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

### Text Preprocessing

In [29]:
# Preprocessing function
def preprocess_text(text):
  """
    This function takes an input text and performs several preprocessing steps to clean and prepare the text for further analysis.

    Args:
        text (str): The input text to be preprocessed.

    Returns:
        str: The preprocessed text after performing lowercase transformation,
             punctuation removal, tokenization, stopword removal, and lemmatization.
  """
  text = text.lower() # Convert the text to lowercase
  text = text.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
  tokens = word_tokenize(text) # Tokenize the text into individual words

  # Remove stopwords and lemmatize words
  stop_words = set(stopwords.words('english'))
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

  # Join the preprocessed tokens back into a coherent text
  review_lematized = " ".join(tokens)

  return review_lematized

In [30]:
# Apply preprocessing to each comment
data["Processed_Comment"] = data["Comment"].apply(preprocess_text)
data.head()

,Comment,Roberta_Sentiment_Scores,Sentiment,Processed_Comment
0,Fun and flexible work environment and an oppor...,"[0.0017704904, 0.049383562, 0.948846]",Positive,fun flexible work environment opportunity lear...
1,"be prepared for constant changes, rules, error...","[0.66478276, 0.3168306, 0.01838666]",Negative,prepared constant change rule error lack support
2,Award-winning training. All the support you ne...,"[0.010903009, 0.2204135, 0.76868343]",Positive,awardwinning training support need havent done...
3,very long hours,"[0.3425702, 0.5902841, 0.067145616]",Negative,long hour
4,Inclusive environment. Good salary and benefit...,"[0.001945477, 0.032334775, 0.96571976]",Positive,inclusive environment good salary benefit mana...


In [31]:
data.loc[90, "Comment"]

'The opportunities to raise through the ladder are low and are extremely tough with very high competition.'

In [32]:
data.loc[90, "Processed_Comment"]

'opportunity raise ladder low extremely tough high competition'

In [33]:
data.loc[90, "Processed_Comment"]

'opportunity raise ladder low extremely tough high competition'

### Topic Modeling using BERTopic

* **UMAP(Uniform Manifold Approximation and Projection)**

UMAP  is a dimensionality reduction technique that's often used in combination with `BERTopic` for visualization purposes. It's used to reduce the high-dimensional vector representations generated by BERT embeddings into a lower-dimensional space while preserving the underlying structure of the data as much as possible.

In [34]:
# Initialize UMAP model for dimensionality reduction
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=45)

# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)

# Apply BERTopic model to transform processed comments
# This step identifies topics in the preprocessed comments and calculates topic probabilities
topics, probabilities = topic_model.fit_transform(data['Processed_Comment'])

In [35]:
# Extract topics - get the list of topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,79,0_work_time_management_good,"[work, time, management, good, day, hour, job,...",[honestly say work culture great want working ...
1,1,20,1_amazon_working_work_team,"[amazon, working, work, team, loved, ive, thin...",[great job loved manger care time work load ba...


In [36]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('work', 0.09768722161098312),
 ('time', 0.06734318099925994),
 ('management', 0.05452003038482862),
 ('good', 0.05088536169250671),
 ('day', 0.04928336464690906),
 ('hour', 0.04807233775373709),
 ('job', 0.03998135561554098),
 ('break', 0.03919419120757278),
 ('like', 0.03839381200331249),
 ('place', 0.03839381200331249)]

In [37]:
# Get top 10 terms for a topic
topic_model.get_topic(1)

[('amazon', 0.20160322111982354),
 ('working', 0.10994436623839304),
 ('work', 0.08267030474268558),
 ('team', 0.07706625332245272),
 ('loved', 0.056783942926480174),
 ('ive', 0.052382862548171163),
 ('thing', 0.04686028366341683),
 ('ppl', 0.04493464630013803),
 ('great', 0.04489768179622335),
 ('people', 0.04489768179622335)]

In [38]:
# Visualize top topic keywords
topic_model.visualize_barchart(top_n_topics=12)

In [39]:
# Visualize similarity using heatmap
topic_model.visualize_heatmap()

In [40]:
# Get the topic predictions
topic_predictions = topic_model.topics_[:]

# Save the predictions in the dataframe
data['topic_prediction'] = topic_predictions

# Create separate columns for each topic
for topic in data['topic_prediction'].unique():
    data[f'Topic {topic}'] = data['topic_prediction'].apply(lambda x: 'Yes' if x == topic else 'No')

# Drop the 'topic_prediction' column if no longer needed
data.drop(columns=['topic_prediction'], inplace=True)

data.head()

,Comment,Roberta_Sentiment_Scores,Sentiment,Processed_Comment,Topic 0,Topic 1
0,Fun and flexible work environment and an oppor...,"[0.0017704904, 0.049383562, 0.948846]",Positive,fun flexible work environment opportunity lear...,Yes,No
1,"be prepared for constant changes, rules, error...","[0.66478276, 0.3168306, 0.01838666]",Negative,prepared constant change rule error lack support,Yes,No
2,Award-winning training. All the support you ne...,"[0.010903009, 0.2204135, 0.76868343]",Positive,awardwinning training support need havent done...,Yes,No
3,very long hours,"[0.3425702, 0.5902841, 0.067145616]",Negative,long hour,Yes,No
4,Inclusive environment. Good salary and benefit...,"[0.001945477, 0.032334775, 0.96571976]",Positive,inclusive environment good salary benefit mana...,Yes,No


In [41]:
# Select and Rename Columns in DataFrame
# We're creating a new DataFrame `df_1` by selecting the important columns from the original `data` DataFrame and renaming them.

# Select columns 'Comment', 'Sentiment', 'Topic 0', and 'Topic 1' from the original DataFrame `data`
df_1 = data[['Comment', 'Sentiment', 'Topic 0', 'Topic 1']]

# We're renaming columns 'Topic 0' and 'Topic 1' to provide more descriptive column names.
df_1 = df_1.rename(columns={'Topic 0':'Topic 0(Work Environment)', 'Topic 1': 'Topic 1(Work Experience)'})
df_1.head()

,Comment,Sentiment,Topic 0(Work Environment),Topic 1(Work Experience)
0,Fun and flexible work environment and an oppor...,Positive,Yes,No
1,"be prepared for constant changes, rules, error...",Negative,Yes,No
2,Award-winning training. All the support you ne...,Positive,Yes,No
3,very long hours,Negative,Yes,No
4,Inclusive environment. Good salary and benefit...,Positive,Yes,No


In [42]:
df_1.to_csv('/content/drive/MyDrive/Data_Science/Coding_Challenge/Answer.csv')

------------

**Author:** *Jammal Adeyemi*